In [1]:
# Disable Cache
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import quantfreedom as qf
import numpy as np
from quantfreedom.backtester.base.base import rsi_below_entries, log_trades_plot
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
pd.options.display.float_format = '{:,.2f}'.format
# qf.clear_cache()

prices = pd.read_csv('30min.csv', index_col='time')
entries = rsi_below_entries(
    timeperiods=[20],
    below_ranges=[50],
    prices=prices.close
)


In [2]:
df, order_rec, cart = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=3.,
    risk_rewards=[3,4,5],
    size_pct=1.,
    tsl_pcts_init=[3, 4, 5],
    tsl_true_or_false=True,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=1,
    tsl_when_pct_from_avg_entry=[3,4,5],
    gains_pct_filter=63,
)


In [3]:
df_or = pd.DataFrame(order_rec)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(20)
    

,order_id,or_set,ind_set,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
0,0,2,0,31,320.70,"10,850.00","10,850.00",NaN,LongEntry,NaN,"1,000.00",NaN,"10,524.50","11,878.61"
1,1,2,0,32,320.70,"10,725.00","10,787.14",NaN,LongEntry,NaN,"1,000.00",NaN,"10,463.52","11,809.79"
2,2,2,0,33,320.70,"10,577.50","10,716.34",NaN,LongEntry,NaN,"1,000.00",NaN,"10,394.85","11,732.28"
3,3,2,0,59,962.09,"10,394.85","10,716.34",1.14,LongTSL,-30.00,970.00,NaN,"10,394.85","11,732.28"
4,4,2,0,60,311.08,"10,448.50","10,448.50",NaN,LongEntry,NaN,970.00,NaN,"10,135.05","11,439.05"
5,5,2,0,61,311.08,"10,484.50","10,466.47",NaN,LongEntry,NaN,970.00,NaN,"10,152.47","11,458.72"
6,6,2,0,62,311.08,"10,507.00","10,479.94",NaN,LongEntry,NaN,970.00,NaN,"10,165.55","11,473.47"
7,7,2,0,407,NaN,NaN,"10,479.94",NaN,MovedTSL,NaN,970.00,NaN,"10,905.84","11,473.47"
8,8,2,0,408,NaN,NaN,"10,479.94",NaN,MovedTSL,NaN,970.00,NaN,"10,926.14","11,473.47"
9,9,2,0,409,NaN,NaN,"10,479.94",NaN,MovedTSL,NaN,970.00,NaN,"10,940.99","11,473.47"


In [4]:
pd.DataFrame(df)


,or_set,ind_set,win_rate,gains_pct,to_the_upside,total_trades,total_BE,total_fees,total_pnl,ending_eq
0,2,0,35.09,63.73,0.04,114.00,0,177.50,637.31,"1,637.31"
1,5,0,42.35,63.61,0.31,85.00,0,97.07,636.09,"1,636.09"
2,8,0,50.68,69.73,0.38,73.00,0,72.66,697.32,"1,697.32"
3,11,0,35.09,79.10,0.13,114.00,0,187.36,790.98,"1,790.98"
4,13,0,45.00,78.92,0.30,100.00,0,122.53,789.21,"1,789.21"
5,14,0,42.35,83.16,0.38,85.00,0,103.52,831.65,"1,831.65"
6,17,0,50.68,65.82,0.34,73.00,0,72.85,658.20,"1,658.20"
7,19,0,38.52,71.74,0.04,135.00,0,219.44,717.37,"1,717.37"
8,20,0,35.09,103.75,0.25,114.00,0,201.92,"1,037.47","2,037.47"
9,22,0,45.00,66.02,0.22,100.00,0,120.50,660.17,"1,660.17"


In [5]:
pd.DataFrame(cart).dropna(axis='columns', thresh=1).T


,0,1,2,3,4,5,6,7,8,9,10,11
order_settings_id,2.00,5.00,8.00,11.00,13.00,14.00,17.00,19.00,20.00,22.00,23.00,26.00
max_equity_risk_pct,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00
risk_rewards,3.00,3.00,3.00,4.00,4.00,4.00,4.00,5.00,5.00,5.00,5.00,5.00
size_pct,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
tsl_based_on,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00
tsl_pcts_init,3.00,4.00,5.00,3.00,4.00,4.00,5.00,3.00,3.00,4.00,4.00,5.00
tsl_trail_by_pct,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
tsl_when_pct_from_avg_entry,5.00,5.00,5.00,5.00,4.00,5.00,5.00,4.00,5.00,4.00,5.00,5.00


In [6]:
log_trades_plot(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_rec[3:15],
)
